In [2]:
# Processing and Displaying run results
import os
import pandas as pd

In [28]:
# Define a function to extract information from the file content
def extract_info_from_file(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    total_time = None
    final_accuracy = None
    for line in reversed(lines):
        if 'Total training time:' in line:
            total_time = float(line.strip().split(': ')[1].split(' ')[0])
        if 'accuracy:' in line:
            final_accuracy = float(line.strip().split(': ')[1].replace('%', ''))
            break  # Stop after the final accuracy is found as it is the first instance from the end

    return total_time, final_accuracy

# Function to parse file name
def parse_filename(file_name):
    parts = file_name.split('-')
    if len(parts) == 5:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3].split('.')[0]  # Remove file extension, prob need to add run number here
    else:
        optimizer, model, dataset, augmentation = None, None, None, None

    return optimizer, model, dataset, augmentation

In [29]:
# Specify the directory
folder_path = 'logs'  # Change 'folder_name' to the name of your folder

# DataFrame to store the results
results_df = pd.DataFrame(columns=['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation', 'Total Time (s)', 'Final Accuracy (%)'])

results_df

,Optimizer,Model Architecture,Dataset,Data Augmentation,Total Time (s),Final Accuracy (%)


In [30]:
# Process each file in the directory
for file in os.listdir(folder_path):
    if file.endswith('.out'):  # Check if the file is a .out file
        file_path = os.path.join(folder_path, file)
        total_time, final_accuracy = extract_info_from_file(file_path)
        optimizer, model, dataset, augmentation = parse_filename(file)
        
        # Append the results to the DataFrame
        results_df = results_df._append({
            'Optimizer': optimizer,
            'Model Architecture': model,
            'Dataset': dataset,
            'Data Augmentation': augmentation,
            'Final Accuracy (%)': final_accuracy,
            'Total Time (s)': total_time
        }, ignore_index=True)

results_df['Total Time (min)'] = results_df['Total Time (s)'] / 60

results_df.head()

/var/folders/qj/v5vpz3c169506rzn5tfshb0h0000gn/T/ipykernel_90521/3471619082.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = results_df._append({


,Optimizer,Model Architecture,Dataset,Data Augmentation,Total Time (s),Final Accuracy (%),Total Time (min)
0,gamsgd,resnet18,cifar100,randaug,18279.216253,79.23,304.653604
1,gamsgd,resnet18,cifar10,autoaug,18346.126232,96.59,305.768771
2,gamsgd,resnet18,cifar10,cutout,18371.204385,96.70,306.186740
3,sgd,resnet18,cifar10,basicaug,4840.658488,95.32,80.677641
4,sgd,resnet18,cifar10,basicaug,4830.081306,95.18,80.501355


In [31]:
grouped_df = results_df.groupby(['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation']).agg({
    'Final Accuracy (%)': ['mean', 'std'],
    'Total Time (min)': ['mean', 'std']
}).reset_index()

# Rename the columns for clarity
grouped_df.columns = [
    'Optimizer', 
    'Model Architecture', 
    'Dataset', 
    'Data Augmentation', 
    'Final Accuracy Mean (%)', 
    'Final Accuracy Std (%)',
    'Total Time Mean (min)', 
    'Total Time Std (min)' 
]

grouped_df

,Optimizer,Model Architecture,Dataset,Data Augmentation,Final Accuracy Mean (%),Final Accuracy Std (%),Total Time Mean (min),Total Time Std (min)
0,gamsgd,resnet18,cifar10,autoaug,96.526000,0.145082,305.029891,1.338794
1,gamsgd,resnet18,cifar10,basicaug,95.929091,0.126211,304.823756,0.718534
2,gamsgd,resnet18,cifar10,cutout,96.582000,0.069410,304.822534,1.392436
3,gamsgd,resnet18,cifar10,randaug,94.871000,0.168024,305.594861,2.092248
4,gamsgd,resnet18,cifar100,autoaug,80.956000,0.167942,305.218879,1.530615
5,gamsgd,resnet18,cifar100,basicaug,79.378000,0.297164,303.943336,0.553741
6,gamsgd,resnet18,cifar100,cutout,79.437000,0.245721,305.533077,1.433217
7,gamsgd,resnet18,cifar100,randaug,79.194000,0.191729,304.897655,0.864304
8,nonaccelgamsgd,resnet18,cifar10,basicaug,95.486667,0.025166,796.528045,1.566712
9,sgd,resnet18,cifar10,autoaug,96.455000,0.135339,84.013850,0.619442


In [32]:
df_sorted = grouped_df.sort_values(by=['Dataset', 'Data Augmentation'])

df_sorted

,Optimizer,Model Architecture,Dataset,Data Augmentation,Final Accuracy Mean (%),Final Accuracy Std (%),Total Time Mean (min),Total Time Std (min)
0,gamsgd,resnet18,cifar10,autoaug,96.526000,0.145082,305.029891,1.338794
9,sgd,resnet18,cifar10,autoaug,96.455000,0.135339,84.013850,0.619442
1,gamsgd,resnet18,cifar10,basicaug,95.929091,0.126211,304.823756,0.718534
8,nonaccelgamsgd,resnet18,cifar10,basicaug,95.486667,0.025166,796.528045,1.566712
10,sgd,resnet18,cifar10,basicaug,95.423000,0.148702,82.969869,1.654304
2,gamsgd,resnet18,cifar10,cutout,96.582000,0.069410,304.822534,1.392436
11,sgd,resnet18,cifar10,cutout,96.349000,0.167030,84.332133,0.589475
3,gamsgd,resnet18,cifar10,randaug,94.871000,0.168024,305.594861,2.092248
4,gamsgd,resnet18,cifar100,autoaug,80.956000,0.167942,305.218879,1.530615
12,sgd,resnet18,cifar100,autoaug,80.085000,0.237966,84.318186,0.455245


In [10]:
from tabulate import tabulate

In [35]:
# Define the custom order for Data Augmentation
augmentation_order = ["basicaug", "cutout", "autoaug", "randaug"]
optimizer_order = ["sgd", "nonaccelgamsgd", "gamsgd"]

# Create a categorical type for Data Augmentation with the specified order
grouped_df["Data Augmentation"] = pd.Categorical(grouped_df["Data Augmentation"], categories=augmentation_order, ordered=True)

# Filter and sort DataFrame to match the desired format
df_sorted = grouped_df.sort_values(by=["Dataset", "Data Augmentation", "Optimizer"])
df_sorted = df_sorted.loc[df_sorted["Data Augmentation"] != "randaug"]
df_cifar10 = df_sorted[df_sorted["Dataset"] == "cifar10"]
df_cifar100 = df_sorted[df_sorted["Dataset"] == "cifar100"]

# Prepare the CIFAR-10 and CIFAR-100 parts of the table
table_cifar10 = df_cifar10.pivot(index="Data Augmentation", columns="Optimizer", values="Final Accuracy Mean (%)").reset_index()
table_cifar100 = df_cifar100.pivot(index="Data Augmentation", columns="Optimizer", values="Final Accuracy Mean (%)").reset_index()

table_cifar10 = table_cifar10[["Data Augmentation"] + optimizer_order]

# Add a new column to distinguish between CIFAR-10 and CIFAR-100
table_cifar10.insert(0, 'Dataset', 'CIFAR-10')
table_cifar100.insert(0, 'Dataset', 'CIFAR-100')

# Combine the tables
table = pd.concat([table_cifar10, table_cifar100], axis=0)

# Print the table
print(tabulate(table, headers='keys', tablefmt='pipe'))

|    | Dataset   | Data Augmentation   |    sgd |   nonaccelgamsgd |   gamsgd |
|---:|:----------|:--------------------|-------:|-----------------:|---------:|
|  0 | CIFAR-10  | basicaug            | 95.423 |          95.4867 |  95.9291 |
|  1 | CIFAR-10  | cutout              | 96.349 |         nan      |  96.582  |
|  2 | CIFAR-10  | autoaug             | 96.455 |         nan      |  96.526  |
|  0 | CIFAR-100 | basicaug            | 79.229 |         nan      |  79.378  |
|  1 | CIFAR-100 | cutout              | 79.38  |         nan      |  79.437  |
|  2 | CIFAR-100 | autoaug             | 80.085 |         nan      |  80.956  |


### Gradient Norm Only Minimization (GNOM)

In [24]:
# Specify the directory
folder_path_gnom = 'logs/GNOM'  

# DataFrame to store the results
results_gnom = pd.DataFrame(columns=['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation', 'Total Time (s)', 'Final Accuracy (%)'])


In [39]:
# Process each file in the directory
for file in os.listdir(folder_path_gnom):
    if file.endswith('.out'):  # Check if the file is a .out file
        file_path = os.path.join(folder_path_gnom, file)
        total_time, final_accuracy = extract_info_from_file(file_path)
        optimizer, model, dataset, augmentation = parse_filename(file)
        
        # Append the results to the DataFrame
        if total_time is not None and final_accuracy is not None and optimizer is not None:
            results_gnom = results_gnom._append({
                'Optimizer': optimizer,
                'Model Architecture': model,
                'Dataset': dataset,
                'Data Augmentation': augmentation,
                'Final Accuracy (%)': final_accuracy,
                'Total Time (s)': total_time
            }, ignore_index=True)

results_gnom['Total Time (min)'] = results_gnom['Total Time (s)'] / 60

results_gnom

,Optimizer,Model Architecture,Dataset,Data Augmentation,Total Time (s),Final Accuracy (%),Total Time (min)
0,gnomsgd,resnet18,cifar100,basicaug,NaN,1.0,NaN
1,gnomsgd,resnet18,cifar10,basicaug,23830.215357,10.0,397.170256
2,gnomsgd,resnet18,cifar100,cutout,NaN,1.0,NaN
3,gnomsgd,resnet18,cifar100,autoaug,23876.951154,1.0,397.949186
4,gnomsgd,resnet18,cifar100,autoaug,23794.963953,1.0,396.582733
...,...,...,...,...,...,...,...
168,gnomsgd,resnet18,cifar100,autoaug,23753.512079,1.0,395.891868
169,gnomsgd,resnet18,cifar10,autoaug,23820.536396,10.0,397.008940
170,gnomsgd,resnet18,cifar10,autoaug,23887.916487,10.0,398.131941
171,gnomsgd,resnet18,cifar100,autoaug,23790.151105,1.0,396.502518
